In [200]:
import pandas as pd
import os
import numpy as np
import re

In [205]:
pc=pd.read_csv('C:/Users/ttodd/OneDrive/Desktop/capstone/final data/PCE_ALL_AREAS.csv')

In [206]:
state_list=["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware",'District of Columbia',"Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]
pc=pc.loc[pc['GeoName'].isin(state_list),:]
pc_percap=pd.read_csv('C:/Users/ttodd/OneDrive/Desktop/capstone/final data/PCPCE_ALL_AREAS.csv')
pc_percap=pc_percap.loc[pc_percap['GeoName'].isin(state_list),:]


In [207]:
pc_2016=pc.loc[:,['GeoName','Description','2011','2012']]
pc_2016['lagged']=pc_2016['2012']-pc_2016['2011']
pc_2016=pc_2016.loc[:,['GeoName','Description','lagged']]
pc_2016_cleaned=pc_2016.pivot(index='GeoName', columns='Description', values='lagged').reset_index().rename_axis(None, axis=1)

pc_2016_cleaned.to_csv('./pc_lagged_2016.csv')

In [219]:
pc_2016_cleaned.columns

Index(['GeoName', '   Clothing and footwear',
       '   Food and beverages purchased for off-premises consumption',
       '   Furnishings and durable household equipment',
       '   Gasoline and other energy goods', '   Motor vehicles and parts',
       '   Other durable goods', '   Other nondurable goods',
       '   Recreational goods and vehicles', '  Durable goods',
       '  Financial services and insurance',
       '  Food services and accommodations', '  Health care',
       '  Household consumption expenditures (for services)',
       '  Housing and utilities', '  Nondurable goods', '  Other services',
       '  Recreation services', '  Transportation services',
       ' Final consumption expenditures of nonprofit institutions serving households (NPISHs)',
       ' Goods', ' Gross output of nonprofit institutions',
       ' Less: Receipts from sales of goods and services by nonprofit institutions',
       ' Services', 'Personal consumption expenditures'],
      dtype='object

In [202]:
#clean personal income data 

In [9]:
variable_list=[]
data_list=['SA1_1929_2017.csv','SA51_1948_2017.csv']
data_path='C:/Users/ttodd/OneDrive/Desktop/capstone/personal_income/spi0318/'
for i in os.listdir(data_path):
    if i.endswith('ALL_AREAS.csv'):
        data_list.append(i)

['SA1_1929_2017.csv',
 'SA51_1948_2017.csv',
 'SA25N_1998_2016__ALL_AREAS.csv',
 'SA25_1969_2001__ALL_AREAS.csv',
 'SA27N_1998_2016__ALL_AREAS.csv',
 'SA27_1969_2001__ALL_AREAS.csv',
 'SA30_1958_2016__ALL_AREAS.csv',
 'SA35_1929_2016__ALL_AREAS.csv',
 'SA40_1958_2016__ALL_AREAS.csv',
 'SA45_1969_2016__ALL_AREAS.csv',
 'SA4_1929_2017__ALL_AREAS.csv',
 'SA50_1948_2016__ALL_AREAS.csv',
 'SA5H_1929_1957__ALL_AREAS.csv',
 'SA5N_1998_2017__ALL_AREAS.csv',
 'SA5_1958_2001__ALL_AREAS.csv',
 'SA6N_1998_2017__ALL_AREAS.csv',
 'SA6_1958_2001__ALL_AREAS.csv',
 'SA7H_1929_1957__ALL_AREAS.csv',
 'SA7N_1998_2017__ALL_AREAS.csv',
 'SA7_1958_2001__ALL_AREAS.csv']

In [225]:
final_result=pd.DataFrame({'state':state_list})
lagged_year_list=['2011','2012']
for i in data_list:
    temp_data=pd.read_csv(data_path+i)
    temp_data_column=temp_data.columns
    temp_data_column=list(temp_data_column)
    final_column_part1=['GeoName','Description']
    final_column_part2=temp_data_column[7:]
    final_column_part2=[x for x in final_column_part2 if x>='1998' and x<='2016']
    final_column=final_column_part1+final_column_part2
    lag_column=final_column_part1+lagged_year_list
    if len(final_column)==21:
        temp_lag_data=temp_data.loc[:,lag_column]
        temp_lag_data.loc[:,lagged_year_list]=temp_lag_data.loc[:,lagged_year_list].apply(pd.to_numeric,errors='coerce')
        temp_lag_data['lagged']=temp_lag_data[lagged_year_list[1]]-temp_lag_data[lagged_year_list[0]]
        temp_lag_data=temp_lag_data.loc[:,final_column_part1+['lagged']]
        temp_lag_data=temp_lag_data.drop_duplicates()
        temp_lag_data=temp_lag_data.dropna(subset=['GeoName','Description'])
        temp_lag_data.replace(0,np.nan)
        temp_lag_data.replace(0.0,np.nan)
        temp_lag_data.dropna(thresh=2)
        drop_column=temp_lag_data['Description'].value_counts()
        drop_column=list(drop_column[drop_column!=60].index)
        temp_lag_data=temp_lag_data.loc[~temp_lag_data['Description'].isin(drop_column),:]
        print(drop_column)
        temp_result=temp_lag_data.pivot(index='GeoName', columns='Description', 
                                        values='lagged').reset_index().rename_axis(None, axis=1)
        temp_result['GeoName']=temp_result['GeoName'].str.replace('*','')
        temp_result=temp_result.loc[temp_result['GeoName'].isin(state_list),:]
        temp_column=temp_result.columns
        
        cleaned_column=[re.sub('([0-9]{1}\/)|(^\s+)','',x) for x in temp_column]
        #if 'Personal income (thousands of dollars)' in cleaned_column:
            #print(i)
        #print(temp_data.loc[temp_data['Description']=='Personal income (thousands of dollars)',lagged_year_list])
        temp_result.columns=cleaned_column
        column_to_use=temp_result.columns.difference(final_result.columns)
        final_result=pd.merge(final_result,temp_result[column_to_use],left_on=final_result.state,right_on=temp_result.GeoName,how='inner')
        final_result=final_result.drop('GeoName',axis=1)
final_result=final_result.T.drop_duplicates().T

[]
[]
[]
[]
[]
[]
[]
[]
[]
['    Motor vehicle licenses', '    Income taxes']
[]
[]
[]


In [214]:
final_result=pd.merge(final_result,pc_2016_cleaned,left_on=final_result.state,right_on=pc_2016_cleaned.GeoName,how='inner')
final_result=final_result.drop('GeoName',axis=1)


In [216]:
final_result

(51, 290)

In [217]:
final_result.to_csv('final_data.csv',index=False)

In [210]:
final_result=final_result.T.drop_duplicates().T

In [211]:
final_result.shape

(51, 710)

In [212]:
final_result.columns

Index(['state', 'Per capita personal income (dollars) _x',
       'Personal income (thousands of dollars)_x', 'Population (persons) _x',
       'Disposable personal income (thousands of dollars)',
       'Per capita disposable personal income (dollars) _x',
       'Apparel manufacturing_x',
       'Beverage and tobacco product manufacturing_x',
       'Chemical manufacturing_x',
       'Computer and electronic product manufacturing_x',
       ...
       '  Nondurable goods', '  Other services', '  Recreation services',
       '  Transportation services',
       ' Final consumption expenditures of nonprofit institutions serving households (NPISHs)',
       ' Goods', ' Gross output of nonprofit institutions',
       ' Less: Receipts from sales of goods and services by nonprofit institutions',
       ' Services', 'Personal consumption expenditures'],
      dtype='object', length=710)

In [203]:
#electricity data

In [174]:
elec=pd.read_csv('C:/Users/ttodd/OneDrive/Desktop/capstone/final data/Average_retail_price_of_electricity.csv',skiprows=4)
#units is cents per killowatthour
elec['GeoName'], elec['type'] = elec['description'].str.split(':', 1).str
elec=elec.drop(['units','source key','description'],axis=1)
elec=elec.dropna(subset=['type'])
elec['GeoName']=elec['GeoName'].str.replace('\s+$','')

In [175]:
elec_state_list=["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware",'District Of Columbia',"Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

elec=elec.loc[elec['GeoName'].isin(elec_state_list),:]